In [1]:
from huggingface_hub import list_models

def list_most_popular(task: str):
    for rank, model in enumerate(
        list_models(filter=task, sort="downloads", direction=-1)
):
        if rank == 5:
            break
        print(f"{model.id}, {model.downloads}\n")

list_most_popular("text-classification")

cardiffnlp/twitter-roberta-base-sentiment-latest, 132942655

mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis, 24024204

smilegate-ai/kor_unsmile, 10856884

distilbert/distilbert-base-uncased-finetuned-sst-2-english, 7985642

cardiffnlp/twitter-roberta-base-sentiment, 2917715



In [2]:
list_most_popular("summarization")

google-t5/t5-small, 3932452

google-t5/t5-base, 3068405

facebook/bart-large-cnn, 2713439

mrm8488/bert2bert_shared-spanish-finetuned-summarization, 841227

philschmid/bart-large-cnn-samsum, 548604



In [3]:
customer_email = """
I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you. I am writing to pour my heart out about the recent unfortunate experience I had with one of your coffee machines that arrived ominously broken, evoking a profound sense of disbelief and despair.

To set the scene, let me paint you a picture of the moment I anxiously unwrapped the box containing my highly anticipated coffee machine. The blatant excitement coursing through my veins could rival the vigorous flow of coffee through its finest espresso artistry. However, what I discovered within broke not only my spirit but also any semblance of confidence I had placed in your esteemed brand.

Imagine, if you can, the utter shock and disbelief that took hold of me as I laid eyes on a disheveled and mangled coffee machine. Its once elegant exterior was marred by the scars of travel, resembling a war-torn soldier who had fought valiantly on the fields of some espresso battlefield. This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection, leaving me emotionally distraught and inconsolable
"""  # created by GPT-3.5

from langchain import HuggingFaceHub

summarizer = HuggingFaceHub(
    repo_id="facebook/bart-large-cnn",
    model_kwargs={"temperature":0, "max_length":180}
)
def summarize(llm, text) -> str:
    return llm(f"Summarize this: {text}!")

summarize(summarizer, customer_email)

/home/mario/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


ValidationError: 1 validation error for HuggingFaceHub
__root__
  Did not find huggingfacehub_api_token, please add an environment variable `HUGGINGFACEHUB_API_TOKEN` which contains it, or pass `huggingfacehub_api_token` as a named parameter. (type=value_error)

In [4]:
from transformers import pipeline

sentiment_model = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/home/mario/.local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [5]:
print(sentiment_model(customer_email))

[{'label': 'LABEL_0', 'score': 0.5822027325630188}]


In [6]:
print(sentiment_model("I am so angry and sad, I want to kill myself!"))

[{'label': 'LABEL_0', 'score': 0.9788626432418823}]


In [7]:
print(sentiment_model("I am elated, I am so happy, this is the best thing that ever happened to me!"))

[{'label': 'LABEL_2', 'score': 0.9926880598068237}]


In [8]:
print(sentiment_model("I don't care. I guess it's ok, or not, I couldn't care one way or the other"))


[{'label': 'LABEL_1', 'score': 0.5958541631698608}]


In [9]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

template = """Given this text, decide what is the issue the customer is concerned about. Valid categories are these:
* product issues
* delivery problems
* missing or late orders
* wrong product
* cancellation request
* refund or exchange
* bad support experience
* no clear reason to be upset

Text: {email}
Category:
"""
prompt = PromptTemplate(template=template, input_variables=["email"])
llm = VertexAI()
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
print(llm_chain.run(customer_email))

/home/mario/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.vertexai.VertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(
/home/mario/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Given this text, decide what is the issue the customer is concerned about. Valid categories are these:
* product issues
* delivery problems
* missing or late orders
* wrong product
* cancellation request
* refund or exchange
* bad support experience
* no clear reason to be upset

Text: 
I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you. I am writing to pour my heart out about the recent unfortunate experience I had with one of your coffee machines that arrived ominously broken, evoking a profound sense of disbelief and despair.

To set the scene, let me paint you a picture of the moment I anxiously unwrapped the box containing my highly anticipated coffee machine. The blatant excitement coursing through my veins could rival the vigorous flow of coffee through its finest espresso artistry. However, what I discovered within broke not only my spi

In [14]:
import sys
from pathlib import Path
# Add the root folder to sys.path
import os

current_working_directory = %pwd
print(current_working_directory)
file_path = current_working_directory


root_path = Path(file_path).resolve().parents[1]  # Adjust the number of parents based on your structure
sys.path.append(str(root_path))

from config import set_environment
from config import set_environment

#from ..config import set_environment

set_environment()

/home/mario/github/generative_ai_with_langchain/notebooks
OPENAI_API_KEY set
HUGGINGFACEHUB_API_TOKEN set
JINACHAT_API_KEY set
REPLICATE_API_TOKEN set
ANTHROPIC_API_KEY set


In [18]:
from langchain import HuggingFaceHub

# facebook model did not work maybe because there are more steps to get access, than just the API key
summarizer = HuggingFaceHub(
    repo_id="google-t5/t5-small",
    #repo_id="facebook/bart-large-cnn",
    model_kwargs={"temperature":0, "max_length":180}
)   

def summarize(llm, text) -> str:
    return llm(f"Summarize this: {text}!")

summarize(summarizer, customer_email)

'Ich hoffe, dass diese E-Mail dich inmitten einer Aura des Verständnisses findet, trotz der schrecklichen Emotionen, die innerhalb von mir wurzeln, wie ich Ihnen schreibe.'

In [19]:
from transformers import pipeline

summarize_model = pipeline(
    task="summarization",
    model="philschmid/bart-large-cnn-samsum"
)

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [20]:
print(summarize_model(customer_email))

[{'summary_text': 'A coffee machine arrived at the store with a broken and disheveled and mangled coffee machine. It shattered his dreams of indulging in daily coffee perfection and left him emotionally distraught and inconsolable. He wrote to you to express his disappointment and express his feelings.'}]
